# Task 5: Verifying a PQC Digital Signature

## Learning Objectives

Upon completing this task, you should be able to:

1.  **Understand the role of a verifier** in the digital signature process.
2.  **Apply** the PQC signature verification process using a signer's public key, the original message, and the digital signature.
3.  **Confirm** message **authenticity** (proof of origin) and **integrity** (proof of no tampering) if the PQC signature is valid.
4.  **Observe and explain** why signature verification fails if the message, signature, or public key is altered or mismatched.
5.  **Appreciate** the robustness of PQC digital signatures in detecting unauthorized modifications.

## Understanding the Task and the Code

As a verifier, you have received a message, a digital signature, and the (claimed) signer's public key. Your goal is to determine if the signature is legitimate for that message and public key.

The verification process takes three inputs:
1.  The **message** that was allegedly signed.
2.  The **digital signature** itself.
3.  The **signer's public key**.

The PQC signature algorithm then performs a mathematical check. If the signature was indeed created for that exact message using the private key corresponding to the provided public key, the verification will succeed. If any of these components are mismatched or if the message was altered after signing, the verification will fail.

**Signature Verification (`on_verify_signature_button_clicked` function):**
*   **Input Validation:** Checks if all required fields are filled.
*   **Decode Inputs:** The Base64 encoded public key and signature are decoded back into raw bytes. The message string is encoded into bytes (UTF-8).
*   **Perform Verification:**
    *   `with oqs.Signature(selected_sig_alg) as sig_verifier:`: Initializes a signature object for the chosen algorithm.
    *   `is_valid = sig_verifier.verify(message_to_verify_bytes, signature_bytes, public_key_sig_bytes)`: This is the core verification operation. It returns `True` if the signature is valid for the given message and public key, and `False` otherwise.
*   **Display Result:** A formatted HTML message indicates whether the signature is VALID or INVALID, along with an explanation of what that means or potential reasons for failure.

**Your Interaction:**

1.  Run the Python code cell below.
2.  From the "Signature Algorithm" dropdown, select the **exact same algorithm** you used in Task 4 when the signature was created.
3.  From your Task 4 output:
    *   Copy the **Base64 encoded Signer's Public Key** and paste it into the "Signer's Public Key (B64)" field.
    *   Carefully copy the **Original Message** (the exact plaintext) and paste or type it into the "Original Message" field.
    *   Copy the **Base64 encoded Digital Signature** and paste it into the "Digital Signature (B64)" field.
4.  Click the "Verify Signature" button.
5.  Observe the result in the "Verification result" area.
6.  **Crucial Experiment:**
    *   After a successful verification, go to the "Original Message" field and change just one character (e.g., add a letter, delete a space, change a word).
    *   Click "Verify Signature" again. You should now see that the signature is INVALID. This shows how digital signatures protect message integrity.

## Understanding the Outcome

*   A **VALID** signature gives you high confidence that the message truly originated from the owner of the private key corresponding to the public key you used, and that the message has not been changed since it was signed.
*   An **INVALID** signature tells you that something is wrong – either the message was tampered with, the signature is not for this message, the public key is incorrect, or the signature itself is corrupted. You should not trust the message in this case.

This task demonstrates a critical component of secure systems: the ability to trust the source and integrity of digital information, even in a post-quantum future.

---
**Now, run the code cell below to perform Task 5.**

In [1]:
import oqs
import ipywidgets as widgets
from IPython.display import display, HTML as IPHTML
import base64

# --- Signature Algorithms (should align with Task 4) ---
# Re-using the dynamic generation logic from Task 4 to ensure consistency
all_supported_sigs_task5 = oqs.get_supported_sig_mechanisms()
standardized_sig_options_task5 = []

# ML-DSA (Dilithium) variants - FIPS 204
dilithium_variants_task5 = {
    "ML-DSA-44": "Dilithium2", "ML-DSA-65": "Dilithium3", "ML-DSA-87": "Dilithium5",
}
for fips_name, common_name in dilithium_variants_task5.items():
    if fips_name in all_supported_sigs_task5:
        standardized_sig_options_task5.append(fips_name)
    elif common_name in all_supported_sigs_task5: # Add common name if FIPS name missing but common one exists
        standardized_sig_options_task5.append(common_name)

# SLH-DSA (SPHINCS+) variants - FIPS 205
sphincs_variants_templates_task5 = [
    ("SLH-DSA-SHA2-", "SPHINCS+-SHA2-", ["128s", "128f", "192s", "192f", "256s", "256f"]),
    ("SLH-DSA-SHAKE-", "SPHINCS+-SHAKE-", ["128s", "128f", "192s", "192f", "256s", "256f"]),
]
sphincs_suffixes_task5 = ["-simple", "-robust", ""] # Ordered preference
for fips_base, liboqs_base, levels in sphincs_variants_templates_task5:
    for level in levels:
        fips_level_name = fips_base + level
        liboqs_level_name_base = liboqs_base + level
        found_sphincs_variant = False
        if fips_level_name in all_supported_sigs_task5: # Direct FIPS name match
            standardized_sig_options_task5.append(fips_level_name)
            found_sphincs_variant = True
        else: # Check common liboqs names with suffixes
            for suffix in sphincs_suffixes_task5:
                liboqs_full_name = liboqs_level_name_base + suffix
                if liboqs_full_name in all_supported_sigs_task5:
                    standardized_sig_options_task5.append(liboqs_full_name)
                    found_sphincs_variant = True
                    break # Found preferred suffix variant for this level
        if found_sphincs_variant:
            continue


supported_sig_algs_task5 = sorted(list(set(standardized_sig_options_task5)))
if not supported_sig_algs_task5: # Absolute fallback
    fallbacks_task5 = ["ML-DSA-65", "Dilithium3", "SLH-DSA-SHA2-128s-simple", "SPHINCS+-SHA2-128s-simple"]
    for fb_alg in fallbacks_task5:
        if fb_alg in all_supported_sigs_task5: supported_sig_algs_task5.append(fb_alg)
    if not supported_sig_algs_task5 and all_supported_sigs_task5: supported_sig_algs_task5 = [all_supported_sigs_task5[0]]
    elif not supported_sig_algs_task5: supported_sig_algs_task5 = ["Dilithium3"]


# --- UI Elements ---
header_task5 = IPHTML("<h2>Task 5: Verifying a PQC Digital Signature</h2>")
description_task5 = IPHTML(f"""
<p>This task demonstrates how to verify a PQC digital signature (created in Task 4) using the signer's public key.
Verification confirms the message's <b>authenticity</b> (it was signed by the holder of the corresponding private key)
and <b>integrity</b> (the message has not been altered since it was signed).</p>
<ol>
    <li>Select the <em>same PQC signature algorithm</em> used in Task 4 to sign the message.</li>
    <li>Paste the <b>Base64 encoded Signer's Public Key</b> (copied from Task 4) into its text area.</li>
    <li>Paste/enter the <b>Original Message</b> (exactly as it was signed in Task 4) into its text area. <i>You can try modifying it slightly after a successful verification to see the verification fail, demonstrating integrity checking!</i></li>
    <li>Paste the <b>Base64 encoded Digital Signature</b> (copied from Task 4) into its text area.</li>
    <li>Click 'Verify Signature'.</li>
</ol>
<p>The result will indicate if the signature is valid or invalid.</p>
<p><i>Available standardized algorithms in your liboqs: {', '.join(supported_sig_algs_task5) if supported_sig_algs_task5 else 'None found, check liboqs.'}</i></p>
""")

sig_alg_dropdown_task5 = widgets.Dropdown(
    options=supported_sig_algs_task5,
    value=supported_sig_algs_task5[0] if supported_sig_algs_task5 else None,
    description='Signature Algorithm:',
    disabled=not supported_sig_algs_task5,
    style={'description_width': 'initial'}
)

sig_public_key_b64_input_task5 = widgets.Textarea(
    value='',
    placeholder="Paste Signer's Base64 Public Key from Task 4 here.",
    description="Signer's Public Key (B64):",
    layout={'width': '95%', 'height': '100px'},
    disabled=False,
    style={'description_width': 'initial'}
)

original_message_input_task5 = widgets.Textarea(
    value='', 
    placeholder="Paste/Enter the EXACT Original Message signed in Task 4.",
    description='Original Message:',
    layout={'width': '95%', 'height': '80px'},
    disabled=False,
    style={'description_width': 'initial'}
)

signature_b64_input_task5 = widgets.Textarea(
    value='',
    placeholder='Paste Base64 Digital Signature from Task 4 here.',
    description='Digital Signature (B64):',
    layout={'width': '95%', 'height': '100px'},
    disabled=False,
    style={'description_width': 'initial'}
)

verify_signature_button_task5 = widgets.Button(
    description='Verify Signature',
    button_style='success', 
    tooltip='Verify the signature against the message and public key',
    icon='check-circle' # FontAwesome 5 icon
)

verification_result_output_task5 = widgets.HTML(
    value="<p style='padding:10px; border:1px solid #ccc; background-color:#f9f9f9;'><i>Verification result will appear here.</i></p>",
    layout={'width': '95%'}
)

status_output_task5 = widgets.Output(layout={'width': '95%'})


# --- Button Click Handler ---
def on_verify_signature_button_clicked(b):
    selected_sig_alg = sig_alg_dropdown_task5.value
    public_key_sig_b64 = sig_public_key_b64_input_task5.value
    message_str = original_message_input_task5.value # This is the message to be verified
    signature_b64 = signature_b64_input_task5.value

    verification_result_output_task5.value = "<p style='padding:10px; border:1px solid #ccc; background-color:#f9f9f9;'><i>Verifying... please wait.</i></p>"

    with status_output_task5:
        status_output_task5.clear_output()

        # Input validation
        if not selected_sig_alg:
            msg = "Error: Please select a signature algorithm."
            print(msg)
            verification_result_output_task5.value = f"<p style='color:red; font-weight:bold;'>{msg}</p>"
            return
        if not public_key_sig_b64.strip():
            msg = "Error: Please paste the Signer's Base64 Public Key."
            print(msg)
            verification_result_output_task5.value = f"<p style='color:red; font-weight:bold;'>{msg}</p>"
            return
        # Message can technically be empty if an empty string was signed.
        # So, no error for empty message_str itself, but signature and public key are essential.
        if not signature_b64.strip():
            msg = "Error: Please paste the Base64 Digital Signature."
            print(msg)
            verification_result_output_task5.value = f"<p style='color:red; font-weight:bold;'>{msg}</p>"
            return

        print(f"Attempting to verify signature for message using {selected_sig_alg}...")
        try:
            # 1. Decode Base64 inputs
            print("Step 1: Decoding Base64 inputs (Public Key and Signature)...")
            public_key_sig_bytes = base64.b64decode(public_key_sig_b64)
            signature_bytes = base64.b64decode(signature_b64)
            print("  Inputs decoded successfully.")
            
            # 2. Encode message to bytes
            # The message from the input field is what we are verifying against.
            message_to_verify_bytes = message_str.encode('utf-8')
            print(f"Step 2: Message to verify (length: {len(message_to_verify_bytes)} bytes): '{message_str[:100]}{'...' if len(message_str)>100 else ''}'")


            # 3. Perform Verification
            print("\nStep 3: Performing signature verification via oqs.Signature.verify()...")
            is_valid = False # Initialize
            with oqs.Signature(selected_sig_alg) as sig_verifier:
                # The verify method takes the message, signature, and public key.
                # It does not require the sig_verifier object to be "primed" with the public key beforehand.
                is_valid = sig_verifier.verify(message_to_verify_bytes, signature_bytes, public_key_sig_bytes)
            
            # 4. Display result
            if is_valid:
                result_html = """
                <div style='padding:10px; border:2px solid green; background-color:#e6ffe6;'>
                    <h3 style='color:green; margin-top:0;'>✅ SIGNATURE IS VALID</h3>
                    <p>The signature correctly verifies against the provided message and public key.</p>
                    <p>This means:</p>
                    <ul>
                        <li><b>Authenticity:</b> The message was signed by the owner of the corresponding private key.</li>
                        <li><b>Integrity:</b> The message has not been altered since it was signed.</li>
                    </ul>
                    <p><i>Try changing a single character in the 'Original Message' field and click 'Verify Signature' again to see it fail.</i></p>
                </div>
                """
                print("  Verification successful: Signature is VALID.")
            else:
                result_html = """
                <div style='padding:10px; border:2px solid red; background-color:#ffe6e6;'>
                    <h3 style='color:red; margin-top:0;'>❌ SIGNATURE IS INVALID</h3>
                    <p>The signature does NOT verify against the provided message and public key.</p>
                    <p>This could be due to:</p>
                    <ul>
                        <li>The message content being different from what was originally signed (check for typos, extra spaces, etc.).</li>
                        <li>The signature itself being incorrect, incomplete, or corrupted.</li>
                        <li>The public key not corresponding to the private key used for signing.</li>
                        <li>The wrong signature algorithm being selected for verification.</li>
                    </ul>
                </div>
                """
                print("  Verification FAILED: Signature is INVALID.")
            
            verification_result_output_task5.value = result_html

        except base64.binascii.Error as e:
            error_msg = f"Base64 Decoding Error: {e}. Please ensure all pasted keys/signatures are valid Base64."
            print(error_msg)
            verification_result_output_task5.value = f"<p style='color:red; font-weight:bold;'>{error_msg}</p>"
        except oqs.OpenSSLError as e: # oqs.Error can also be a base, OpenSSLError is more specific for crypto issues
            error_msg = f"OQS Cryptographic Error: {e}. This often indicates a fundamental issue with the key, signature, or algorithm mismatch during verification."
            print(error_msg)
            verification_result_output_task5.value = f"<p style='color:red; font-weight:bold;'>{error_msg}</p>"
        except Exception as e:
            error_msg = f"An unexpected error occurred: {e}"
            print(error_msg)
            import traceback
            traceback.print_exc() # For more detailed debugging in notebook console
            verification_result_output_task5.value = f"<p style='color:red; font-weight:bold;'>{error_msg}</p>"

verify_signature_button_task5.on_click(on_verify_signature_button_clicked)

# --- Display UI ---
display(header_task5)
display(description_task5)

main_ui_container_task5 = widgets.VBox([
    sig_alg_dropdown_task5,
    sig_public_key_b64_input_task5,
    original_message_input_task5,
    signature_b64_input_task5,
    verify_signature_button_task5,
    widgets.HTML("<hr style='margin-top: 20px; margin-bottom:10px;'>"), # Separator before result
    verification_result_output_task5,
    status_output_task5
], layout={'width': '100%'})

display(main_ui_container_task5)

/usr/local/lib/python3.13/site-packages/oqs/__init__.py:1: UserWarning: liboqs version (major, minor) 0.13.1-dev differs from liboqs-python version 0.12.0
  from oqs.oqs import (
